In [5]:
from nltk.tokenize import sent_tokenize, TweetTokenizer

text = "This is a great value for the money . We purchased this as a back up computer after our more expensive HP needed to be repaired . This is a great computer . We have n't had any problems with it at all . The body is a bit cheaply made so it will be interesting to see how long it holds up . Overall though , for the money spent it 's a great deal ."

tokens = TweetTokenizer().tokenize(text)
print(tokens)
print(len(tokens))

['This', 'is', 'a', 'great', 'value', 'for', 'the', 'money', '.', 'We', 'purchased', 'this', 'as', 'a', 'back', 'up', 'computer', 'after', 'our', 'more', 'expensive', 'HP', 'needed', 'to', 'be', 'repaired', '.', 'This', 'is', 'a', 'great', 'computer', '.', 'We', 'have', "n't", 'had', 'any', 'problems', 'with', 'it', 'at', 'all', '.', 'The', 'body', 'is', 'a', 'bit', 'cheaply', 'made', 'so', 'it', 'will', 'be', 'interesting', 'to', 'see', 'how', 'long', 'it', 'holds', 'up', '.', 'Overall', 'though', ',', 'for', 'the', 'money', 'spent', 'it', "'", 's', 'a', 'great', 'deal', '.']
78


In [4]:
def is_whitespace(c):
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False

paragraph_text = text
doc_tokens = []
char_to_word_offset = []
prev_is_whitespace = True
for c in paragraph_text:
    if is_whitespace(c):
        prev_is_whitespace = True
    else:
        if prev_is_whitespace:
            doc_tokens.append(c)
        else:
            doc_tokens[-1] += c
        prev_is_whitespace = False
    char_to_word_offset.append(len(doc_tokens) - 1)

print(doc_tokens)
print(len(doc_tokens))

['This', 'is', 'a', 'great', 'value', 'for', 'the', 'money', '.', 'We', 'purchased', 'this', 'as', 'a', 'back', 'up', 'computer', 'after', 'our', 'more', 'expensive', 'HP', 'needed', 'to', 'be', 'repaired', '.', 'This', 'is', 'a', 'great', 'computer', '.', 'We', 'have', "n't", 'had', 'any', 'problems', 'with', 'it', 'at', 'all', '.', 'The', 'body', 'is', 'a', 'bit', 'cheaply', 'made', 'so', 'it', 'will', 'be', 'interesting', 'to', 'see', 'how', 'long', 'it', 'holds', 'up', '.', 'Overall', 'though', ',', 'for', 'the', 'money', 'spent', 'it', "'s", 'a', 'great', 'deal', '.']
77


In [2]:
from transformers import BertModel, BertTokenizer

bert = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

sentence = "I like this weather. It is sunny!"

